In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Step 1: 構造化された出力のための Pydantic モデルを定義する

これらのモデルは、エージェントが返す**スキーマ**を定義します。Pydantic を用いた `response_format` により以下が保証されます：
- ✅ 型安全なデータ抽出
- ✅ 自動検証
- ✅ 自由形式テキストの応答による解析エラーなし
- ✅ フィールドに基づく簡単な条件付きルーティング


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ステップ2: ホテル予約ツールの作成

このツールは**availability_agent**が部屋の空き状況を確認するために呼び出します。`@ai_function`デコレーターを使用して以下を行います：
- Python関数をAI呼び出し可能なツールに変換
- LLM用のJSONスキーマを自動生成
- パラメータ検証の処理
- エージェントによる自動呼び出しを可能にする

このデモでは：
- **ストックホルム、シアトル、東京、ロンドン、アムステルダム** → 部屋あり ✅
- **その他の都市すべて** → 部屋なし ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ステップ3: ルーティングのための条件関数を定義する

これらの関数はエージェントの応答を検査し、ワークフローでどのパスを進むかを決定します。

**重要なパターン:**
1. メッセージが `AgentExecutorResponse` かどうかを確認する
2. 構造化された出力（Pydanticモデル）を解析する
3. ルーティングを制御するために `True` または `False` を返す

ワークフローはこれらの条件を**エッジ**で評価し、次にどのエグゼキューターを呼び出すかを決定します。


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Step 4: カスタム表示エグゼキューターの作成

エグゼキューターは変換や副作用を行うワークフローのコンポーネントです。`@executor` デコレーターを使って、最終結果を表示するカスタムエグゼキューターを作成します。

**重要な概念:**
- `@executor(id="...")` - 関数をワークフローエグゼキューターとして登録する
- `WorkflowContext[Never, str]` - 入力/出力の型ヒント
- `ctx.yield_output(...)` - 最終的なワークフロー結果を返す


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## ステップ5: 環境変数の読み込み

LLMクライアントを設定します。この例は以下で動作します:
- **GitHub Models** (GitHubトークンを使った無料プラン)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## ステップ6: 構造化された出力を持つAIエージェントの作成

私たちは **3つの専門エージェント** を作成し、それぞれを `AgentExecutor` でラップします。

1. **availability_agent** - ツールを使ってホテルの空き状況を確認します
2. **alternative_agent** - 空室がない場合に代替都市を提案します
3. **booking_agent** - 空室がある場合に予約を促します

**主な特徴:**
- `tools=[hotel_booking]` - エージェントにツールを提供します
- `response_format=PydanticModel` - 構造化されたJSON出力を強制します
- `AgentExecutor(..., id="...")` - ワークフローで使用するためにエージェントをラップします


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ステップ7: 条件付きエッジでワークフローを構築する

ここでは、`WorkflowBuilder` を使用して条件付きルーティングを持つグラフを構築します:

**ワークフローの構造:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**主なメソッド:**
- `.set_start_executor(...)` - エントリーポイントを設定する
- `.add_edge(from, to, condition=...)` - 条件付きエッジを追加する
- `.build()` - ワークフローを確定する


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## ステップ8: テストケース1を実行 - 空室なしの都市（パリ）

パリ（当シミュレーションでは空室がない）でホテルをリクエストして、**空室なし**のパスをテストしましょう。


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Step 9: テストケース 2 を実行 - 空室ありの都市（ストックホルム）

次に、ストックホルムのホテルをリクエストして、**空室**のパスをテストしましょう（シミュレーションでは部屋がある都市です）。


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## 重要なポイントと次のステップ

### ✅ 学んだこと：

1. **WorkflowBuilderパターン**
   - `.set_start_executor()`でエントリーポイントを定義する
   - `.add_edge(from, to, condition=...)`で条件付きルーティングを行う
   - `.build()`を呼び出してワークフローを確定する

2. **条件付きルーティング**
   - 条件関数は`AgentExecutorResponse`を検査する
   - 構造化出力を解析してルーティング判断を行う
   - エッジを有効にする場合は`True`、スキップする場合は`False`を返す

3. **ツールの統合**
   - Python関数をAIツールに変換するには`@ai_function`を使用する
   - エージェントは必要に応じてツールを自動的に呼び出す
   - ツールはエージェントが解析できるJSONを返す

4. **構造化出力**
   - 安全なデータ抽出にはPydanticモデルを使う
   - エージェントを作成するときに`response_format=MyModel`を設定する
   - `Model.model_validate_json()`でレスポンスを解析する

5. **カスタムエグゼキューター**
   - `@executor(id="...")`でワークフローコンポーネントを作成する
   - エグゼキューターはデータ変換や副作用の処理を行える
   - `ctx.yield_output()`でワークフローの結果を出力する

### 🚀 実際の応用例：

- **旅行予約**：空き状況の確認、代替案の提案、選択肢の比較
- **カスタマーサービス**：問題タイプ、感情、優先度によるルーティング
- **Eコマース**：在庫確認、代替品の提案、注文処理
- **コンテンツモデレーション**：有害度スコアやユーザーフラグによるルーティング
- **承認ワークフロー**：金額、ユーザーの役割、リスクレベルによるルーティング
- **多段階処理**：データ品質や完了度に基づくルーティング

### 📚 次のステップ：

- より複雑な条件（複数の基準）を追加する
- ワークフロー状態管理を用いたループ処理の実装
- 再利用可能なコンポーネントのためのサブワークフローの追加
- 実際のAPI（ホテル予約、在庫管理システム）との統合
- エラーハンドリングとフォールバックパスの追加
- 組み込みの可視化ツールでワークフローを視覚化する


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責事項**：
本書類はAI翻訳サービス「Co-op Translator」（https://github.com/Azure/co-op-translator）を使用して翻訳されました。正確性の確保に努めておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。原文の言語によるオリジナル文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じたいかなる誤解や解釈違いについても、一切の責任を負いかねます。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
